In [1]:
# ==========================================
# 0. INSTALL DEPENDENCIES
# ==========================================
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install datasets sentence-transformers faiss-cpu tqdm

import os
import json
import pandas as pd
from unsloth import FastLanguageModel
from google.colab import drive
import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-uyyr48ml/unsloth_19437218ce4a4fbd86437e2bd8537609
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-uyyr48ml/unsloth_19437218ce4a4fbd86437e2bd8537609
  Resolved https://github.com/unslothai/unsloth.git to commit b96a04c17bc6bcb5522eafb17adc2b104be38f99
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.8/180.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 136.8 MB/s eta 0:00:00

In [2]:
# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Hugging Face Cache to Drive (Persistent)
DRIVE_CACHE = "/content/drive/MyDrive/LLM project/Cache/HF"
os.makedirs(DRIVE_CACHE, exist_ok=True)

os.environ["HF_HOME"] = f"{DRIVE_CACHE}/hf_home"
os.environ["HF_HUB_CACHE"] = f"{DRIVE_CACHE}/hf_hub"
os.environ["HF_DATASETS_CACHE"] = f"{DRIVE_CACHE}/datasets"
os.environ["TRANSFORMERS_CACHE"] = f"{DRIVE_CACHE}/transformers"
os.environ["SENTENCE_TRANSFORMERS_HOME"] = f"{DRIVE_CACHE}/sentence_transformers"

from huggingface_hub import login
# login(token="YOUR_HF_TOKEN")

# Paths
model_id = "unsloth/gemma-2-9b-it-bnb-4bit"
input_path = '/content/task-a-en.tsv'
output_file = "/content/outputs_gemma_dual_rag.jsonl"

# Config for Retrieval
N_WIKI_DOCS = 20000  # Stable size
N_JOKE_DOCS = 5000   # Stable size


Mounted at /content/drive


In [3]:
# ==========================================
# 2. LOAD DUAL RAG SYSTEM (Retriever)
# ==========================================
class DualRetriever:
    def __init__(self, n_wiki=20000, n_jokes=5000, seed=42):
        print("Initializing Dual Retriever...")
        self.seed = seed

        # Cache folder (Drive)
        cache_dir = "/content/drive/MyDrive/LLM project/Cache/Retriever"
        os.makedirs(cache_dir, exist_ok=True)

        # Embedding model (CPU)
        self.encoder_name = "mixedbread-ai/mxbai-embed-large-v1"
        print("Loading embedding model on CPU (Safe Mode)...")
        self.encoder = SentenceTransformer(self.encoder_name, device="cpu")

        # -------------------------
        # A) Wikipedia (embedded) - Sliced Load
        # -------------------------
        print("Loading Wikipedia (embedded) slice...")
        wiki_full = load_dataset("not-lain/wikipedia", revision="embedded", split="train")
        actual_n_wiki = min(int(n_wiki), len(wiki_full))
        print(f"Using {actual_n_wiki} wiki docs (from {len(wiki_full)} total)")

        wiki_ds = load_dataset(
            "not-lain/wikipedia",
            revision="embedded",
            split=f"train[:{actual_n_wiki}]"
        ).shuffle(seed=seed)

        self.wiki_texts = [str(x) for x in wiki_ds["text"]]
        self.wiki_embs = np.asarray(wiki_ds["embeddings"], dtype=np.float32)
        self.wiki_embs /= (np.linalg.norm(self.wiki_embs, axis=1, keepdims=True) + 1e-12)

        # -------------------------
        # B) Jokes (style) - Cached
        # -------------------------
        actual_n_jokes = int(n_jokes)
        emb_tag = self.encoder_name.replace("/", "_")
        jokes_emb_path = os.path.join(cache_dir, f"jokes_embs_{actual_n_jokes}_{emb_tag}.npy")
        jokes_txt_path = os.path.join(cache_dir, f"jokes_texts_{actual_n_jokes}.npy")

        if os.path.exists(jokes_emb_path) and os.path.exists(jokes_txt_path):
            print(f" Found cached jokes index. Loading from Drive...")
            self.joke_embs = np.load(jokes_emb_path)
            self.joke_texts = np.load(jokes_txt_path, allow_pickle=True).tolist()
        else:
            print(" Cache not found. Building jokes index ONCE...")
            jokes_ds = load_dataset("weixingxing/short-jokes-dataset", split="train")
            actual_n_jokes = min(actual_n_jokes, len(jokes_ds))
            print(f"Using {actual_n_jokes} jokes (from {len(jokes_ds)} total)")

            jokes_ds = jokes_ds.shuffle(seed=seed).select(range(actual_n_jokes))

            joke_texts = []
            for ex in jokes_ds:
                s = ex.get("Joke", "")
                s = "" if s is None else str(s).strip()
                if s and s.lower() != "nan":
                    joke_texts.append(s)
            self.joke_texts = joke_texts

            print("Embedding jokes (CPU)...")
            self.joke_embs = self.encoder.encode(
                self.joke_texts,
                convert_to_numpy=True,
                show_progress_bar=True,
                batch_size=64,
                normalize_embeddings=True,
            ).astype(np.float32)

            print(f" Saving jokes index to Drive: {cache_dir}")
            np.save(jokes_emb_path, self.joke_embs)
            np.save(jokes_txt_path, np.array(self.joke_texts, dtype=object))

        print("Dual RAG Ready!")

    def retrieve(self, query, k_wiki=1, k_jokes=5):
        try:
            query = str(query).strip()
            if not query:
                return "Fact:\n\nStyle Inspiration:\n-"

            q_vec = self.encoder.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)[0]

            # Wiki
            sims_wiki = self.wiki_embs @ q_vec
            # Use argpartition for speed
            top_w_idx = np.argpartition(-sims_wiki, kth=min(k_wiki, len(sims_wiki)-1))[:k_wiki]
            # Sort only top k
            top_w_idx = top_w_idx[np.argsort(-sims_wiki[top_w_idx])]
            wiki_str = self.wiki_texts[top_w_idx[0]][:350] if len(top_w_idx) else "" # Limit chars

            # Jokes (retrieve multiple for style pack)
            sims_j = self.joke_embs @ q_vec
            top_j_idx = np.argpartition(-sims_j, kth=min(k_jokes, len(sims_j)-1))[:k_jokes]
            top_j_idx = top_j_idx[np.argsort(-sims_j[top_j_idx])]

            joke_samples = [self.joke_texts[i][:150] for i in top_j_idx] # Limit chars per joke
            joke_str = "\n- ".join(joke_samples)

            return f"Fact: {wiki_str}\n\nStyle Inspiration:\n- {joke_str}"

        except Exception as e:
            print(f"Retrieval failed: {e}")
            return ""

# Initialize Retriever
retriever = DualRetriever(n_wiki=N_WIKI_DOCS, n_jokes=N_JOKE_DOCS)

Initializing Dual Retriever...
Loading embedding model on CPU (Safe Mode)...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

data/train-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/231657 [00:00<?, ? examples/s]

Using 5000 jokes (from 231657 total)
Embedding jokes (CPU)...


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

💾 Saving jokes index to Drive: /content/drive/MyDrive/LLM project/Cache/Retriever
Dual RAG Ready!


In [4]:
# ==========================================
# 3. LOAD GEMMA MODEL
# ==========================================
print(f"Loading Gemma from {model_id}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype = None,
)
FastLanguageModel.for_inference(model)

Loading Gemma from unsloth/gemma-2-9b-it-bnb-4bit...
==((====))==  Unsloth 2026.1.2: Fast Gemma2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm):

In [5]:
# ==========================================
# 4. GENERATION LOOP
# ==========================================
# Read Input
df = pd.read_csv(input_path, sep='\t')
data = df.to_dict('records')

# Resume Check
processed_ids = set()
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                saved_item = json.loads(line)
                processed_ids.add(saved_item['id'])
            except: pass
    print(f"Resuming... Found {len(processed_ids)} jokes.")

print("Starting Dual-RAG Generation with Gemma...")

for row in tqdm(data, desc="Generating Jokes"):
    current_id = row['id']
    if current_id in processed_ids: continue

    # Parse Input
    headline_val = str(row.get('headline', "-")).strip()
    w1_val = str(row.get('word1', "-")).strip()
    w2_val = str(row.get('word2', "-")).strip()

    # --- RAG RETRIEVAL (DIFFERENT LEVELS) ---

    # LEVEL 1: NO RAG (Just Gemma)
    context_no_rag = ""

    # LEVEL 2: WIKI ONLY (Fact)
    context_wiki = ""
    # LEVEL 3: DUAL (Fact + Style)
    context_dual = ""

    if headline_val != "-" and headline_val != "" and headline_val.lower() != "nan":
        # Retrieve full Dual context
        full_context_str = retriever.retrieve(headline_val)

        # Split Context if possible
        if "Style Inspiration:" in full_context_str:
            parts = full_context_str.split("Style Inspiration:")
            fact_part = parts[0].replace("Fact:", "").strip()
            style_part = parts[1].strip()

            context_wiki = f"Fact: {fact_part}"
            context_dual = full_context_str
        else:
            context_wiki = full_context_str
            context_dual = full_context_str

        input_type = "headline"
        input_content = headline_val

        # Base Prompt Template Generator
        def make_prompt(ctx):
            if ctx:
                return f"""### Instruction
You are a witty, cynical stand-up comedian.
Use the following background information ONLY if it helps inspire a joke. Otherwise, ignore it.

Background Info:
{ctx}

Headline: "{headline_val}"

Task: Write EXACTLY ONE punchy joke (1-2 sentences) about this headline.
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.

### Response
Joke:"""
            else:
                 return f"""### Instruction
You are a witty, cynical stand-up comedian.
Headline: "{headline_val}"

Task: Write EXACTLY ONE punchy joke (1-2 sentences) about this headline.
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.

### Response
Joke:"""

    else:
        # Words Case
        real_w1 = w1_val if w1_val != "-" else "something"
        real_w2 = w2_val if w2_val != "-" else "random"
        query = f"{real_w1} {real_w2}"

        full_context_str = retriever.retrieve(query)

        if "Style Inspiration:" in full_context_str:
            parts = full_context_str.split("Style Inspiration:")
            fact_part = parts[0].replace("Fact:", "").strip()
            style_part = parts[1].strip()

            context_wiki = f"Fact: {fact_part}"
            context_dual = full_context_str
        else:
            context_wiki = full_context_str
            context_dual = full_context_str

        input_type = "word-pair"
        input_content = f"{real_w1} {real_w2}"

        def make_prompt(ctx):
            if ctx:
                 return f"""### Instruction
You are a witty, cynical stand-up comedian.
Use the following background information ONLY if it helps inspire a joke. Otherwise, ignore it.

Background Info:
{ctx}

Task: Write EXACTLY ONE punchy joke (1-2 sentences) connecting: "{real_w1}" and "{real_w2}".
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.

### Response
Joke:"""
            else:
                 return f"""### Instruction
You are a witty, cynical stand-up comedian.
Task: Write EXACTLY ONE punchy joke (1-2 sentences) connecting: "{real_w1}" and "{real_w2}".
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.

### Response
Joke:"""

    # --- GENERATE (OPTIMIZED) ---

    # Helper to generate
    def generate_one(prompt_str):
        inputs = tokenizer([prompt_str], return_tensors="pt").to("cuda")
        with torch.inference_mode(): # Memory optimization
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                do_sample = True,
                max_new_tokens = 64, # Reduced from 128
                temperature = 0.9,
                top_p = 0.9,
                repetition_penalty = 1.2
            )
        dec_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response" in dec_text:
            resp = dec_text.split("### Response")[-1]
        else:
            resp = dec_text
        if "Joke:" in resp:
            final = resp.split("Joke:")[-1].strip()
        else:
            final = resp.strip()
        final = final.split("\n\n")[0].strip().strip('"').strip("'")

        # Gemma artifacts
        if final.lower().startswith("model"):
            final = final[5:].strip().lstrip(":").strip()
        if final.lower().startswith("assistant"):
            final = final[9:].strip().lstrip(":").strip()

        if not final or final.lower() in ["model", "assistant"]:
            final = dec_text.replace(prompt_str, "").strip().split("\n\n")[0]

        return final

    prompt_final = make_prompt(context_dual)
    joke_final = generate_one(prompt_final)

    # Periodic Cache Clear
    if len(processed_ids) % 50 == 0:
        torch.cuda.empty_cache()

    # Save Combined
    result_entry = {
        "id": current_id,
        "type": input_type,
        "input_original": input_content,
        "context_retrieved": context_dual,
        "generated_joke": joke_final
    }

    with open(output_file, "a", encoding='utf-8') as f:
        f.write(json.dumps(result_entry, ensure_ascii=False) + "\n")

print("Finished Dual-RAG Generation!")


Starting Dual-RAG Generation with Gemma...


Generating Jokes: 100%|██████████| 1200/1200 [55:30<00:00,  2.78s/it]

Finished Dual-RAG Generation!
